In [1]:
import matrix_ant_colony_optimization as aco
import random 
import numpy as np
import threading
import queue

import utils.prog_lineraire as pl
import utils.generator_matrix as gm
import utils.dataset as dataset
import pandas as pd

In [2]:
def generate_nodes(number,max_x=1080,max_y=720,seed=None):
    # create number of nodes with coordinates of x,y  with max value of 600
    random.seed(seed)
    nodes = []
    for i in range(number):
        nodes.append([random.randint(0.1*max_y,max_x-0.1*max_y),random.randint(0.1*max_y,max_y-0.1*max_y)])
    return nodes

def distance(x1,y1,x2,y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

def coordinats_to_matrice(nodes):
    # create a matrice with the nodes coordinates
    matrice = np.zeros((len(nodes),len(nodes)))
    for x in range(len(nodes)):
        for y in range(len(nodes)):
            matrice[x][y] = distance(nodes[x][0],nodes[x][1],nodes[y][0],nodes[y][1])
    return matrice

In [13]:

parameters = {
    "nbr_ant":[50,100,150,200],#np.arange(25,500,25)
    "a":np.arange(0,6.5,.5),
    "b":np.arange(1,6.5,.5),
    "p":np.arange(0,1.05,.05),
    "nb_iter_max":[1000],
    "time_max":[120],
    "nbr_random_search":[40],
    "nbr_loops":[2],
    "nbr_workers":[20],
}

constraint = {
    "enabled":False,
    "start":4,
    "city":18,
    "inferior_dist":1000,
    "superior_dist":2000,
}


In [4]:
datasets = []
for x in range(10):

    ds = dataset.import_dataset("dataset/dataset"+str(x))
    datasets.append(
        {
            "matrice":ds[0],
            "cities":ds[1],
            "coords":ds[2],
            "borne_inf":pl.get_borne_inf(ds[0],ds[1])
        }
    )


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/colin/.local/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/1c7bf64524a542f7b3ab2abc7014a3a8-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/1c7bf64524a542f7b3ab2abc7014a3a8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 65 COLUMNS
At line 2066 RHS
At line 2127 BOUNDS
At line 2528 ENDATA
Problem MODEL has 60 rows, 400 columns and 820 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2548.56 - 0.00 seconds
Cgl0002I 20 variables fixed
Cgl0004I processed model has 40 rows, 380 columns (380 integer (380 of which binary)) and 760 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 2548.56
Cbc0038I Before mini branch and bound, 380 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did n

# Optimisation
Comment evaluer mes algorithmes ?
Comment faire pour comparer les algorithmes ?

Déterminer la borne pour chacun des dataset

Tester une combiaison de paramètres sur chaque dataset pour obtenir une moyenne et l'ecart type:


1. Prendre une combinaison de paramètres
2. Tester les paramètres sur chaque dataset 10 fois.
   1. Récupérer la moyenne et l'écart type des 10 essais sur chaque dataset.
1. Evaluer l'effcatité d'une combinaison.
   






In [5]:
def random_parameters(parameters):
    """Return random parameters in parameters

    Args:
        parameters (array): array of parameters

    Returns:
        dict: return dict of random parameters
    """
    return {key:random.choice(parameters[key]) for key in parameters}


In [6]:
def worker(q):
    while not q.empty():
        item = q.get()
        print("Starting ",item[4],"/",item[5])
        ds = item[0]
        parameter =item[1]
        results = item[2]
        constraint = item[3]
        aco.aco_algorithm(ds,parameter,results,constraint)
        print("Ending ",item[4],"/",item[5])
        q.task_done()

In [7]:
def GridSearchCV(parameters,datasets,constraint):

    iteration_to_do = len(datasets)*parameters["nbr_loops"][0]*parameters["nbr_random_search"][0]

    for rdm_search in range(parameters["nbr_random_search"][0]):
        
        parameter = random_parameters(parameters)

        print("Random search : {}/{}".format(rdm_search+1,parameters["nbr_random_search"][0]))
        print("Parameters : {}".format(parameter))

        results = []    
        q = queue.Queue()

        


        for idx,ds in enumerate(datasets):
            for loop in range(parameter["nbr_loops"]):

                iteration_done = len(datasets)*parameter["nbr_loops"]*rdm_search + idx*parameter["nbr_loops"] + loop
                q.put([ds,parameter,results,constraint,iteration_done,iteration_to_do])

        for y in range (parameter["nbr_workers"]):
            thread = threading.Thread(target=worker, args=(q,),daemon=True)
            thread.start()

        q.join()


        mean_score = np.mean([result[2] for result in results])
        standard_deviation = np.std([result[2] for result in results])

        df = pd.read_json("dataset/results_rdm_search_cv.json")
        row = {"nbr_ant":parameter["nbr_ant"],"a":parameter["a"],"b":parameter["b"],"p":parameter["p"],"mean_score":mean_score,"std_score":standard_deviation}
        df = df.append(row,ignore_index=True)
        df.to_json("dataset/results_rdm_search_cv.json")

        


In [14]:

GridSearchCV(parameters,datasets,constraint)

Random search : 1/40
Parameters : {'nbr_ant': 100, 'a': 6.0, 'b': 5.0, 'p': 0.4, 'nb_iter_max': 1000, 'time_max': 120, 'nbr_random_search': 40, 'nbr_loops': 2, 'nbr_workers': 20}
Starting  0 / 800
Starting  1 / 800
Starting  2 / 800
Starting  3 / 800
Starting  4 / 800
Starting  5 / 800
Starting  6 / 800
Starting  7 / 800
Starting  8 / 800
Starting  9 / 800
Starting  10 / 800
Starting  11 / 800
Starting  12 / 800
Starting  13 / 800
Starting  14 / 800
Starting  15 / 800
Starting  16 / 800
Starting  17 / 800
Starting  18 / 800
Starting  19 / 800
Ending  4 / 800
Ending  0 / 800
Ending  3 / 800
Ending  1 / 800
Ending  19 / 800
Ending  2 / 800
Ending  18 / 800
Ending  17 / 800
Ending  13 / 800
Ending  6 / 800
Ending  16 / 800
Ending  7 / 800
Ending  15 / 800
Ending  5 / 800
Ending  8 / 800
Ending  10 / 800
Ending  14 / 800
Ending  9 / 800
Ending  11 / 800
Ending  12 / 800
Random search : 2/40
Parameters : {'nbr_ant': 150, 'a': 2.5, 'b': 3.0, 'p': 0.0, 'nb_iter_max': 1000, 'time_max': 120, 'n

/tmp/ipykernel_255310/1513704113.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


Starting  36 / 800
Starting  37 / 800
Starting  38 / 800
Starting  39 / 800
Ending  25 / 800
Ending  27 / 800
Ending  34 / 800
Ending  26 / 800
Ending  29 / 800
Ending  22 / 800
Ending  30 / 800
Ending  32 / 800
Ending  31 / 800
Ending  33 / 800
Ending  37 / 800
Ending  39 / 800
Ending  35 / 800
Ending  36 / 800
Ending  38 / 800
Ending  28 / 800
Ending  21 / 800
Ending  20 / 800
Ending  23 / 800
Ending  24 / 800
Random search : 3/40
Parameters : {'nbr_ant': 50, 'a': 2.0, 'b': 3.0, 'p': 0.45, 'nb_iter_max': 1000, 'time_max': 120, 'nbr_random_search': 40, 'nbr_loops': 2, 'nbr_workers': 20}
Starting  40 / 800
Starting  41 / 800
Starting  42 / 800
Starting  43 / 800
Starting  44 / 800


/tmp/ipykernel_255310/1513704113.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


Starting  45 / 800
Starting  46 / 800
Starting  47 / 800
Starting  48 / 800
Starting  49 / 800
Starting  50 / 800
Starting  51 / 800
Starting  52 / 800
Starting  53 / 800
Starting  54 / 800
Starting  55 / 800
Starting  56 / 800
Starting  57 / 800
Starting  58 / 800
Starting  59 / 800
Ending  46 / 800
Ending  51 / 800
Ending  44 / 800
Ending  47 / 800
Ending  55 / 800
Ending  41 / 800
Ending  49 / 800
Ending  59 / 800
Ending  48 / 800
Ending  54 / 800
Ending  50 / 800
Ending  53 / 800
Ending  56 / 800
Ending  42 / 800
Ending  58 / 800
Ending  40 / 800
Ending  43 / 800
Ending  45 / 800
Ending  57 / 800
Ending  52 / 800
Random search : 4/40
Parameters : {'nbr_ant': 200, 'a': 5.0, 'b': 1.0, 'p': 0.25, 'nb_iter_max': 1000, 'time_max': 120, 'nbr_random_search': 40, 'nbr_loops': 2, 'nbr_workers': 20}
Starting  60 / 800
Starting  61 / 800
Starting  62 / 800
Starting  63 / 800
Starting  64 / 800
Starting  65 / 800
Starting  66 / 800
Starting  67 / 800


...

/tmp/ipykernel_255310/1513704113.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


Starting  765 / 800
Starting  766 / 800
Starting  767 / 800
Starting  768 / 800
Starting  769 / 800
Starting  770 / 800
Starting  771 / 800
Starting  772 / 800
Starting  773 / 800
Starting  774 / 800
Starting  775 / 800
Starting  776 / 800
Starting  777 / 800
Starting  778 / 800
Starting  779 / 800
Ending  762 / 800
Ending  760 / 800
Ending  767 / 800
Ending  766 / 800
Ending  768 / 800
Ending  769 / 800
Ending  765 / 800
Ending  774 / 800
Ending  770 / 800
Ending  764 / 800
Ending  772 / 800
Ending  773 / 800
Ending  761 / 800
Ending  775 / 800
Ending  771 / 800
Ending  779 / 800
Ending  776 / 800
Ending  763 / 800
Ending  777 / 800
Ending  778 / 800
Random search : 40/40
Parameters : {'nbr_ant': 100, 'a': 2.5, 'b': 3.5, 'p': 0.30000000000000004, 'nb_iter_max': 1000, 'time_max': 120, 'nbr_random_search': 40, 'nbr_loops': 2, 'nbr_workers': 20}
Starting  780 / 800
Starting  781 / 800
Starting  782 / 800
Starting  783 / 800
Starting  784 / 800
Starting  785 / 800
Starting  786 / 800
Star

/tmp/ipykernel_255310/1513704113.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


Starting  798 / 800
Starting  799 / 800
Ending  793 / 800
Ending  787 / 800
Ending  795 / 800
Ending  791 / 800
Ending  788 / 800
Ending  781 / 800
Ending  799 / 800
Ending  790 / 800
Ending  796 / 800
Ending  786 / 800
Ending  785 / 800
Ending  783 / 800
Ending  780 / 800
Ending  784 / 800
Ending  797 / 800
Ending  794 / 800
Ending  789 / 800
Ending  782 / 800
Ending  798 / 800
Ending  792 / 800


/tmp/ipykernel_255310/1513704113.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row,ignore_index=True)


In [9]:
#df =   pd.DataFrame(columns=["nbr_ant","a","b","p","mean_score","std_score"])
#df.to_json("dataset/results_rdm_search_cv.json")

In [10]:
df = pd.read_json("dataset/results_rdm_search_cv.json").sort_values(by="mean_score",ascending=False)
print(df)

   nbr_ant     a     b     p  mean_score  std_score
9      100  1.75  3.00  0.85   87.641312   7.509831
1      100  1.75  1.50  0.40   87.454844   8.432383
6      100  2.75  3.50  0.30   83.389666   7.328652
4      100  3.00  2.25  0.90   77.908321   6.863758
5      100  4.00  3.75  0.70   74.483239   5.848813
2      100  0.25  3.50  0.10   74.297171   6.448374
8      100  0.50  3.00  0.85   72.432712   3.538297
7      100  0.00  3.25  0.75   70.966022   5.393167
0      100  2.00  1.50  0.00   57.320503   5.747976
3      100  0.50  1.00  0.40   54.486925   4.639958
